In [1]:
from helium import *
import pandas as pd
import time
import json
import scraping_amazon

# ouverture du dataset des sous=categories

In [2]:
amazon = pd.read_csv('datasets/amazon_categories.csv')
amazon.head(2)

,cat1,cat2,cat2_links
0,Amazon Renewed,Renewed Camera & Photo,https://www.amazon.fr/gp/bestsellers/amazon-re...
1,Amazon Renewed,Renewed Computers & Accessories,https://www.amazon.fr/gp/bestsellers/amazon-re...


In [5]:
amazon.cat1.unique()

array(['Amazon Renewed', 'Animalerie', 'Appareils Amazon et Accessoires',
       'Applis et Jeux', 'Auto et Moto', 'Beauté et Parfum',
       'Bébé et Puériculture', 'Boutique cartes cadeaux',
       'Boutique Kindle', 'Bricolage', 'CD et Vinyles',
       'Climate Pledge Friendly', 'Commerce, Industrie et Science',
       'Cuisine et Maison', 'DVD et Blu-ray', 'Epicerie',
       'Fournitures de bureau', 'Gros électroménager', 'High-Tech',
       'Hygiène et Santé', 'Informatique',
       'Instruments de musique et Sono', 'Jardin', 'Jeux et Jouets',
       'Jeux vidéo', 'Livres', 'Logiciels', 'Luminaires et Éclairage',
       'Mode', 'Nouveautés dénichées', 'Produits Handmade',
       'Sports et Loisirs'], dtype=object)

In [3]:
webdriver = start_firefox(amazon.cat2_links[0])

In [16]:
count_requests = 0

category = 'Bébé et Puériculture'
df_filter_category = amazon.query("cat1 == 'Bébé et Puériculture' ")

list_of_df = []

for cat1, cat2, links in zip(df_filter_category['cat1'], df_filter_category['cat2'], df_filter_category['cat2_links']):
    go_to(links)
    
    pages_to_scrap = find_all(S('//html/body/div[1]/div[2]/div/div/div[1]/div/div/div[2]/div[2]/ul/li/a'))
    pages_links = [links.web_element.get_attribute('href') for links in pages_to_scrap[:2]]
    
    # scrap sur les 2 pages de la sous-categorie :
    for page in pages_links:
        count_requests += 1
        go_to(page)
        
        # scroll pour activer les 50 articles par page
        for i in range(4):
            scroll_down(2000)
            time.sleep(2)
        
        # --------------------------------------------------------------->  recherche des images grand format:
        search_pictures = find_all(S('//html/body/div[1]/div[2]/div/div/div[1]/div/div/div[2]/div[1]/div[1]/div/div/div/div[2]/div/a/div/img'))
        pictures = [list(json.loads(pic.web_element.get_attribute('data-a-dynamic-image')))[-1] for pic in search_pictures]
        
        # --------------------------------------------------------------->  recherche des noms de produits & liens produits :
        search_titles = find_all(S('//html/body/div[1]/div[2]/div/div/div[1]/div/div/div[2]/div[1]/div[1]/div/div/div/div[2]/div/div/div/a'))
        titles = [title.web_element.text for title in search_titles]
        products_links = [link.web_element.get_attribute('href') for link in search_titles]
        
        # --------------------------------------------------------------->  travail sur les prix et note / nb_votes :
        search_infos = find_all(S('//*[@id="gridItemRoot"]'))
        prices = [scraping_amazon.scrap_price(price) for price in search_infos]
        nb_votes = [scraping_amazon.scrap_nb_votes(votes) for votes in search_infos]
        votes = [scraping_amazon.scrap_votes(votes) for votes in search_infos]
        # --------------------------------------------------------------->  récupération des categories et sous-categories :
        first_category = [cat1 for i in range(len(search_titles))]
        second_category = [cat2 for i in range(len(search_titles))]
        
        # --------------------------------------------------------------->  création du df temporaire :
        df = pd.DataFrame({
            'first_cat': first_category,
            'second_cat': second_category,
            'titles': titles,
            'product_link' : products_links,
            'pictures' : pictures,
            'prices' : prices,
            'nb_votes' : nb_votes,
            'votes' : votes           
                        })
        list_of_df.append(df)
        sep = "***-----------------------------***"
        print(sep)
        print(f"scraping de la page N° {count_requests} sur {len(df_filter_category['cat2'])*2} : à la page {page}, pour la catégorie {cat1}, et la sous-catégorie : {cat2} DONE !")
        print(sep)
    time.sleep(5)
    
        


***-----------------------------***
scraping de la page N° 1 sur 34 : à la page https://www.amazon.fr/gp/bestsellers/baby/3966365031/ref=zg_bs_pg_1_baby?ie=UTF8&pg=1, pour la catégorie Bébé et Puériculture, et la sous-catégorie : Apprentissage de la propreté DONE !
***-----------------------------***
***-----------------------------***
scraping de la page N° 2 sur 34 : à la page https://www.amazon.fr/gp/bestsellers/baby/3966365031/ref=zg_bs_pg_2_baby?ie=UTF8&pg=2, pour la catégorie Bébé et Puériculture, et la sous-catégorie : Apprentissage de la propreté DONE !
***-----------------------------***
***-----------------------------***
scraping de la page N° 3 sur 34 : à la page https://www.amazon.fr/gp/bestsellers/baby/3966233031/ref=zg_bs_pg_1_baby?ie=UTF8&pg=1, pour la catégorie Bébé et Puériculture, et la sous-catégorie : Cadeaux de naissance et souvenirs DONE !
***-----------------------------***
***-----------------------------***
scraping de la page N° 4 sur 34 : à la page https://w

TimeoutException: Message: Navigation timed out after 300000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:740:5
bail@chrome://remote/content/marionette/sync.sys.mjs:211:19


In [14]:
full_amazon_data = pd.concat(list_of_df)
full_amazon_data

,first_cat,second_cat,titles,product_link,pictures,prices,nb_votes,votes
0,Cuisine et Maison,Ameublement et décoration,Amazon Basics Drap-housse en microfibre Bleu m...,https://www.amazon.fr/AmazonBasics-Drap-housse...,https://images-eu.ssl-images-amazon.com/images...,9.21,134006,4.5
1,Cuisine et Maison,Ameublement et décoration,Protege Matelas 160x200 cm Imperméable Oeko TE...,https://www.amazon.fr/Prot%C3%A8ge-Matelas-Imp...,https://images-eu.ssl-images-amazon.com/images...,19.99,30948,4.4
2,Cuisine et Maison,Ameublement et décoration,Utopia Bedding Housse de Couette 220x240 cm av...,https://www.amazon.fr/Utopia-Bedding-Parure-Li...,https://images-eu.ssl-images-amazon.com/images...,20.99,41790,4.5
3,Cuisine et Maison,Ameublement et décoration,"douceur d'intérieur, Drap Housse (140 x 190 cm...",https://www.amazon.fr/Douceur-dInt%C3%A9rieur-...,https://images-eu.ssl-images-amazon.com/images...,9.70,26394,4.3
4,Cuisine et Maison,Ameublement et décoration,PHYTOSUN AROMS - Huile Essentielles Eucalyptus...,https://www.amazon.fr/Phytosun-Ar%C3%B4ms-Euca...,https://images-eu.ssl-images-amazon.com/images...,3.20,10891,4.6
...,...,...,...,...,...,...,...,...
33,Cuisine et Maison,Vaisselle et arts de la table,Wanapix | Gobelet en Plastique pour Enfants | ...,https://www.amazon.fr/Wanapix-Plastique-Person...,https://images-eu.ssl-images-amazon.com/images...,14.95,406,4.4
34,Cuisine et Maison,Vaisselle et arts de la table,"Lanjue 6 Pièces Couverts Enfant en INOX, 2 x F...",https://www.amazon.fr/Lanjue-Couverts-Fourchet...,https://images-eu.ssl-images-amazon.com/images...,11.49,780,4.7
35,Cuisine et Maison,Vaisselle et arts de la table,"TASSE EN PLASTIQUE DE 390 ML SANS BPA, PASSANT...",https://www.amazon.fr/PLASTIQUE-PASSANT-MICRO-...,https://images-eu.ssl-images-amazon.com/images...,2.99,312,4.2
36,Cuisine et Maison,Vaisselle et arts de la table,CROWNLY CRYSTAL® Cadeau Homme 40 Ans Idee Cade...,https://www.amazon.fr/CROWNLY-CRYSTAL%C2%AE-Ca...,https://images-eu.ssl-images-amazon.com/images...,21.90,30,4.6


# export du dataframe par catégorie

In [15]:
full_amazon_data.to_csv(f'datasets/amazon_{category}.csv', index=False, encoding='utf-8')